DB_RAW
 └─ SCHEMA_RAW
      ├─ CUSTOMERS_RAW
      └─ ORDERS_RAW

DB_STAGING
 └─ SCHEMA_STAGING
      ├─ CUSTOMERS_STAGE
      └─ ORDERS_STAGE

DB_ANALYTICS
 └─ SCHEMA_ANALYTICS
      ├─ CUSTOMERS
      └─ ORDERS

#### Instalar SnowSQL en la VM

- wget https://sfc-repo.snowflakecomputing.com/snowsql/bootstrap/1.2/linux_x86_64/snowsql-1.2.24-linux_x86_64.bash

- bash snowsql-1.2.24-linux_x86_64.bash

#### Crear warehouse de laboratorio

CREATE WAREHOUSE WH_LAB
  WAREHOUSE_SIZE = 'XSMALL'
  AUTO_SUSPEND = 60
  AUTO_RESUME = TRUE
  INITIALLY_SUSPENDED = TRUE;

#### crear databases
CREATE DATABASE DB_RAW;
CREATE DATABASE DB_STAGING;
CREATE DATABASE DB_ANALYTICS;

#### Crear esquemas
CREATE SCHEMA DB_RAW.SCHEMA_RAW;
CREATE SCHEMA DB_STAGING.SCHEMA_STAGING;
CREATE SCHEMA DB_ANALYTICS.SCHEMA_ANALYTICS;

#### ver esquemas creados:
USE SCHEMA DB_ANALYTICS.SCHEMA_ANALYTICS;

### Crear el Rol de analista
CREATE ROLE ANALYST;
GRANT USAGE ON DATABASE DB_ANALYTICS TO ANALYST;
GRANT USAGE ON SCHEMA DB_ANALYTICS.SCHEMA_ANALYTICS TO ANALYST;
GRANT SELECT ON ALL TABLES IN SCHEMA DB_ANALYTICS.SCHEMA_ANALYTICS TO ANALYST;

-- Permitir uso de la base de datos
GRANT USAGE ON DATABASE DB_RAW TO ROLE RAW_LAB;

-- Permitir uso del esquema
GRANT USAGE ON SCHEMA DB_RAW.SCHEMA_RAW TO ROLE RAW_LAB;

-- Permitir trabajar con todas las tablas actuales del esquema
GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA DB_RAW.SCHEMA_RAW TO ROLE RAW_LAB;

-- Opcional: para que también aplique a futuras tablas
GRANT SELECT, INSERT, UPDATE, DELETE ON FUTURE TABLES IN SCHEMA DB_RAW.SCHEMA_RAW TO ROLE RAW_LAB;


#### ver los permisos de de un rol
SHOW GRANTS TO ROLE  RAW_LAB;
SHOW GRANTS TO ROLE ANALYST;

#### asignar permisos a un rol
GRANT SELECT ON ALL TABLES IN SCHEMA DB_ANALYTICS.SCHEMA_ANALYTICS TO ROLE ANALYST;

GRANT SELECT ON FUTURE TABLES IN SCHEMA DB_ANALYTICS.SCHEMA_ANALYTICS TO ROLE ANALYST;

GRANT USAGE ON WAREHOUSE WH_LAB TO ROLE ANALYST;

### Crear ususarios y roles analytics
CREATE USER JuanMiranda
    PASSWORD = 'contraseñajuan'
    DEFAULT_ROLE = ANALYST
    MUST_CHANGE_PASSWORD = TRUE
    DEFAULT_WAREHOUSE = WH_LAB
    DEFAULT_NAMESPACE = DB_ANALYTICS.SCHEMA_ANALYTICS;



GRANT ROLE ANALYTICS TO USER JuanMiranda;





#### crear una table de pruebas 
CREATE OR REPLACE TABLE contactos_prueba (
    id INT AUTOINCREMENT PRIMARY KEY,
    first_name STRING,
    last_name STRING,
    email STRING,
    phone STRING,
    country STRING,
    fecha_creacion TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP
);

#### maquina virtual para pruebas de tráfico
az vm create \
  --resource-group rg-data-lab \
  --name vm-datalab-b2ms \
  --image Ubuntu2404 \
  --size Standard_B2ms \
  --vnet-name vnet-data-lab \
  --subnet subnet-app \
  --admin-username labuser \
  --generate-ssh-keys \
  --assign-identity  \
  --storage-sku StandardSSD_LRS \

#### conectarse a la maquina virtual

  
az network bastion ssh --name bastion-lab --resource-group rg-data-lab --target-resource-id $(az vm show -g rg-data-lab -n vm-datalab-2 --query id -o tsv) --auth-type ssh-key --username labuser --ssh-key "C:\Users\ANDRESFELIPEARDILALO\.ssh\id_rsa"

#### E comando para redimensionar la maquina virtual si llega a ser necesario


#### debe estar deallocated 
az vm deallocate --resource-group rg-data-lab --name vm-datalab-2

az vm resize \
  --resource-group rg-data-lab \
  --name vm-datalab-2 \
  --size Standard_D2s_v3


#### CONFIGURAR EL GRUPO DE RECURSOS (VNet, Key Vault, Entra ID, NSG, VM)



#### consumo por fechas de azure
- az account list --output table


az account set --subscription "<SUBSCRIPTION_ID>"

az consumption usage list --subscription db059ab1-ccee-408a-9ad2-8e1d6fddddf4 --start-date 2025-09-29 --end-date 2025-10-03 --output table

az consumption usage list \
  --subscription <SUBSCRIPTION_ID> \
  --start-date 2025-09-01 \
  --end-date 2025-09-29 \
  -o table

#### GESTIÓN DE RECURSOS PARA ACESSO LA MAUINA VIRTUAL
Usuarios pertenecientes al grupo Entra ID "Lab_DataAnalytics" se conectan mediante Azure Bastion usando sus credeneciales(correo y contraseña) a la maquina virtual vm-datalab-2

#### se requiere la extensión AADSSHLoginForLinux instalada en la maquina virtual
az vm extension set \
  --publisher Microsoft.Azure.ActiveDirectory \
  --name AADSSHLoginForLinux \
  --resource-group rg-data-lab \
  --vm-name vm-datalab-2

#### obtener ID del grupo 
- az ad group show --group Lab_DataAnalytics --query id -o tsv
7eedd430-6e28-49f8-8a75-4636731fc311

#### obtener los miembros del grupo Entra ID
- az ad group member list --group Lab_DataAnalytics --query "[].{name:displayName, userPrincipalName:userPrincipalName}" -o table

- az ad group show --group Lab_DataAnalytics --query "id" -o tsv
7eedd430-6e28-49f8-8a75-4636731fc311

#### ver permisos que tiene el grupo
- az role assignment list --assignee 7eedd430-6e28-49f8-8a75-4636731fc311 --scope $(az vm show -g rg-data-lab -n vm-datalab-2 --query id -o tsv) -o table

#### asignar el rol para entrar a la maquina virtual

- az role assignment create --assignee 7eedd430-6e28-49f8-8a75-4636731fc311 --role "Virtual Machine User Login" --scope $(az vm show --resource-group rg-data-lab --name vm-datalab-2 --query id -o tsv)

#### se requiere adicionar el rol Reader para que pueda usar la maquina virtual 
- az role assignment create --assignee 7eedd430-6e28-49f8-8a75-4636731fc311 --role "Reader" --scope /subscriptions/db059ab1-ccee-408a-9ad2-8e1d6fddddf4/resourceGroups/rg-data-lab

#### camando de acceso a la maquina virtual desde azure cli
az network bastion ssh  --name bastion-lab --resource-group rg-data-lab  --target-resource-id $(az vm show -g rg-data-lab -n vm-datalab-2 --query id -o tsv) --auth-type AAD



#### INTEGRACIÓN SNOWFLAKE + KEY VAULT 

#### Asignar permisos de key vault al grupo de Lab_DataAnalytics
-- los usuarios se conectan a la VM, necesitan leer secretos desde Key Vault (credenciales de Snowflake, cuentas, tokens, etc.).

requiere el objectId

-  az role assignment create --assignee 7eedd430-6e28-49f8-8a75-4636731fc311 --role "Key Vault Secrets User" --scope $(az keyvault show --name kv-datalab2025 --resource-group rg-data-lab --query id -o tsv)



#### Configurar Network Policies en Snowflake
- El objetivo es controlar desde dónde se puede conectar a Snowflake, asegurando que solo la VM o la red corporativa tenga acceso

- listar 
- SHOW NETWORK POLICIES;
- DESCRIBE NETWORK POLICY DATA_GROUP_1;

#### Crear una política de red en Snowflake que permita solo rangos IP específicos (por ejemplo, de la VM o VPN corporativa)

curl -s ifconfig.me
172.191.113.147 (maquina virtual vm-data-lab-2)

IP de mi equipo propia de admin de cuenta
'177.253.98.42',

(Daniel)
177.253.153.82


(Juan)
177.253.153.82

IP del servicio de autenticaicón de snowflake
'34.120.55.22',
'34.121.90.77',
'35.239.180.99',

IP conectores de power bi y servicios de microsoft
44.203.133.160/28, 3.145.219.176/28, 35.89.45.128/28



#### Network Policy
CREATE NETWORK POLICY LAB_DATA
    ALLOWED_IP_LIST = ('172.191.113.147',
                       '177.253.98.42',
                       '192.168.0.119',
                       '34.120.55.22',
                       '34.121.90.77',
                       '35.239.180.99',
                       '44.203.133.160/28',
                       '3.145.219.176/28',
                       '35.89.45.128/28'
    )
    BLOCKED_IP_LIST = ('0.0.0.0/0');

#### actualizar ip
ALTER NETWORK POLICY LAB_DATA
  SET ALLOWED_IP_LIST = ()

#### Listar las IP: 
DESC NETWORK POLICY LAB_DATA;


#### Asociar la política de red a los usuarios de Snowflake o roles específicos

ALTER USER afelipe2025 SET NETWORK_POLICY = lab_vm_only;
ALTER USER CARLOSRUEDA SET NETWORK_POLICY =LAB_DATA;
ALTER USER JUANMIRANDA SET NETWORK_POLICY =LAB_DATA;


ALTER ROLE lab_data_analytics SET NETWORK_POLICY = lab_vm_only;